In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila/Liudmila_data/sales (2).xlsx")

In [3]:
df.head()

,product name,unique_id,Сегмент рынка,Категория товара,date,weight,price,cost price,promo action,exp,...,рейтинг товаров,количество просмотров,конкуренция,month,day_of_week,sales_3m_avg,sales_6m_avg,sales_12m_avg,discount,seasonal_discount
0,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,уход за телом,соль,2021-06-15,2000,363,78.0,нет,36,...,2.5,5000,низкая,6,1,7.0,7.0,7.0,7,0
1,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,уход за телом,соль,2021-06-22,2000,363,78.0,нет,36,...,2.5,5000,низкая,6,1,5.5,5.5,5.5,7,0
2,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,уход за телом,соль,2021-06-29,2000,363,78.0,нет,36,...,2.5,5000,низкая,6,1,8.7,8.7,8.7,7,0
3,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,уход за телом,соль,2021-07-06,2000,363,78.0,нет,36,...,2.5,5000,низкая,7,1,7.0,7.0,7.0,7,0
4,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,уход за телом,соль,2021-07-13,2000,363,78.0,нет,36,...,2.5,5000,низкая,7,1,7.7,6.8,6.8,7,0


In [4]:
series = df[['unique_id', 'date', 'y']]

In [5]:
series.to_csv('time_series', index=False)

In [6]:
data = series

In [7]:
data

,unique_id,date,y
0,9,2021-06-15,7
1,9,2021-06-22,4
2,9,2021-06-29,15
3,9,2021-07-06,2
4,9,2021-07-13,6
...,...,...,...
30523,423,2024-02-13,250
30524,423,2024-02-20,250
30525,423,2024-02-27,250
30526,423,2024-03-05,500


In [8]:
import numpy as np
from statsmodels.tsa.stattools import adfuller

# Process the data
summary = []

# Loop through each unique_id
for unique_id, group in data.groupby('unique_id'):
    # Handle outliers using IQR method
    Q1 = group['y'].quantile(0.25)
    Q3 = group['y'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    group['y'] = np.where(group['y'] < lower_bound, lower_bound, group['y'])
    group['y'] = np.where(group['y'] > upper_bound, upper_bound, group['y'])
    
    # Check stationarity using Augmented Dickey-Fuller test
    adf_result = adfuller(group['y'])
    is_stationary = adf_result[1] < 0.05
    summary.append({"unique_id": unique_id, "is_stationary": is_stationary})

# Create a summary DataFrame
stationarity_summary = pd.DataFrame(summary)

# Count stationary and non-stationary series
stationary_count = stationarity_summary['is_stationary'].sum()
non_stationary_count = len(stationarity_summary) - stationary_count

stationarity_summary, stationary_count, non_stationary_count


(     unique_id  is_stationary
 0            9          False
 1           10           True
 2           11          False
 3           12           True
 4           13           True
 ..         ...            ...
 207        389          False
 208        390          False
 209        391          False
 210        392          False
 211        423          False
 
 [212 rows x 2 columns],
 78,
 134)

In [9]:
stationarity_summary


,unique_id,is_stationary
0,9,False
1,10,True
2,11,False
3,12,True
4,13,True
...,...,...
207,389,False
208,390,False
209,391,False
210,392,False


### Итак, сколько у нас стационарных и нестационарных рядов:

* Стационарных рядов: 78
* Нестационарных рядов: 134

In [10]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Process the data for all IDs
summary_season_trend_all = []

# Loop through each unique_id
for unique_id, group in data.groupby('unique_id'):
    # Perform first-order differencing
    group['y_diff'] = group['y'].diff().dropna()

    # Seasonal decomposition
    result = seasonal_decompose(group['y_diff'].dropna(), period=12, model='additive', extrapolate_trend='freq')
    has_seasonality = np.max(result.seasonal) - np.min(result.seasonal) > 0
    has_trend = np.max(result.trend.dropna()) - np.min(result.trend.dropna()) > 0
    summary_season_trend_all.append({"unique_id": unique_id, "has_seasonality": has_seasonality, "has_trend": has_trend})

# Create a summary DataFrame
season_trend_summary_all = pd.DataFrame(summary_season_trend_all)

# Count series with trend and seasonality
trend_count_all = season_trend_summary_all['has_trend'].sum()
seasonality_count_all = season_trend_summary_all['has_seasonality'].sum()
no_trend_count = len(season_trend_summary_all) - trend_count_all
no_seasonality_count = len(season_trend_summary_all) - seasonality_count_all

season_trend_summary_all, trend_count_all, seasonality_count_all, no_trend_count, no_seasonality_count


(     unique_id  has_seasonality  has_trend
 0            9             True       True
 1           10             True       True
 2           11             True       True
 3           12             True       True
 4           13             True       True
 ..         ...              ...        ...
 207        389             True       True
 208        390             True       True
 209        391             True       True
 210        392             True       True
 211        423             True       True
 
 [212 rows x 3 columns],
 212,
 212,
 0,
 0)

Применение метода сезонного декомпозирования
Метод seasonal_decompose работает следующим образом:

Сезонность: Сравнивается амплитуда сезонной компоненты. Если разница между максимальным и минимальным значением значительна, считается, что ряд имеет сезонность.
Тренд: Аналогично, если амплитуда трендовой компоненты значительна, считается, что ряд имеет тренд.

Итоги:
Рядов с трендом и сезонностью: 212
Рядов без тренда и сезонности: 0

Вывод:
Все ряды (212) имеют тренд и сезонность. Таким образом, нет ни одного ряда, который не имеет тренда и сезонности.

### Этап 3: Избавление от тренда и сезонности, применение преобразования Бокса-Кокса

* Избавление от тренда и сезонности: Удалим тренд и сезонность с использованием сезонного декомпозирования.
* Преобразование Бокса-Кокса: Применим преобразование Бокса-Кокса к результатам.

In [11]:
from scipy.stats import boxcox

# Process the data
summary_boxcox = []

# Loop through each unique_id
for unique_id, group in data.groupby('unique_id'):
    # Perform first-order differencing
    group['y_diff'] = group['y'].diff().dropna()

    # Seasonal decomposition
    result = seasonal_decompose(group['y_diff'].dropna(), period=12, model='additive', extrapolate_trend='freq')
    detrended = group['y_diff'] - result.seasonal - result.trend
    detrended.dropna(inplace=True)

    # Box-Cox Transformation
    detrended_boxcox, lambda_val = boxcox(detrended + abs(detrended.min()) + 1)
    summary_boxcox.append({"unique_id": unique_id, "lambda": lambda_val})

# Create a summary DataFrame
boxcox_summary = pd.DataFrame(summary_boxcox)

# Display the resulting summary
boxcox_summary


,unique_id,lambda
0,9,1.068202
1,10,1.161877
2,11,1.146129
3,12,1.067669
4,13,1.297839
...,...,...
207,389,1.135850
208,390,1.163371
209,391,1.178638
210,392,1.161786


Вывод:
В результате избавления от тренда и сезонности и применения преобразования Бокса-Кокса, были найдены оптимальные значения параметра λ для каждого ряда. Эти значения находятся в диапазоне примерно от 1.06 до 1.30, что означает, что преобразование привело данные к более нормальному распределению.

### Этап 4: Тест Дарбина-Уотсона на автокорреляцию

* Для выявления автокорреляции в данных мы применим тест Дарбина-Уотсона.

#### Тест Дарбина-Уотсона: 
* Для каждого ряда мы вычислим статистику Дарбина-Уотсона.
* Сводная таблица: Создадим сводную таблицу с результатами и выведем выводы о том, в скольких рядах есть автокорреляция.

In [12]:
from statsmodels.stats.stattools import durbin_watson

# Process the data
summary_autocorrelation = []

# Loop through each unique_id
for unique_id, group in data.groupby('unique_id'):
    # Perform first-order differencing
    group['y_diff'] = group['y'].diff().dropna()
    
    # Seasonal decomposition
    result = seasonal_decompose(group['y_diff'].dropna(), period=12, model='additive', extrapolate_trend='freq')
    detrended = group['y_diff'] - result.seasonal - result.trend
    detrended.dropna(inplace=True)

    # Box-Cox Transformation
    detrended_boxcox, lambda_val = boxcox(detrended + abs(detrended.min()) + 1)

    # Durbin-Watson test
    dw_stat = durbin_watson(detrended_boxcox)
    autocorrelation = dw_stat < 1.5 or dw_stat > 2.5
    summary_autocorrelation.append({"unique_id": unique_id, "durbin_watson": dw_stat, "autocorrelation": autocorrelation})

# Create a summary DataFrame
autocorrelation_summary = pd.DataFrame(summary_autocorrelation)

# Count series with autocorrelation
autocorrelation_count = autocorrelation_summary['autocorrelation'].sum()

autocorrelation_summary, autocorrelation_count


(     unique_id  durbin_watson  autocorrelation
 0            9       0.078733             True
 1           10       0.079672             True
 2           11       0.076883             True
 3           12       0.108359             True
 4           13       0.113761             True
 ..         ...            ...              ...
 207        389       0.071359             True
 208        390       0.069436             True
 209        391       0.073333             True
 210        392       0.070524             True
 211        423       0.072441             True
 
 [212 rows x 3 columns],
 212)

Вывод:
Все ряды (212) имеют значительную автокорреляцию, как выявлено тестом Дарбина-Уотсона. Автокорреляция является серьезной проблемой для временных рядов, так как это может негативно повлиять на качество прогнозирования.

In [16]:
import numpy as np
import pandas as pd
from math import sqrt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import boxcox, shapiro
import warnings
warnings.filterwarnings("ignore")

# Функция для проверки стационарности
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    return dftest[1]

# Функция для подбора и обучения модели ARIMA на основе критерия AIC
def fit_model(ts):
    best_aic = np.inf
    best_order = None
    best_mdl = None

    pq_range = range(3)
    d_range = range(3)

    for p in pq_range:
        for d in d_range:
            for q in pq_range:
                try:
                    tmp_mdl = ARIMA(ts, order=(p, d, q)).fit()
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (p, d, q)
                        best_mdl = tmp_mdl
                except:
                    continue

    return best_aic, best_order, best_mdl

# Dataframe для хранения результатов
results_df = pd.DataFrame(columns=['unique_id', 'aic', 'order', 'mse', 'mae', 'r2', 'error_rate', 'has_trend', 'has_seasonality', 'is_normal'])

# Обработка каждого уникального идентификатора продукта
unique_ids = data['unique_id'].unique()
for uid in tqdm(unique_ids, desc="Processing Unique IDs"):
    ts_data = data[data['unique_id'] == uid].set_index('date')['y']
    ts_data.index = pd.DatetimeIndex(ts_data.index).to_period('W')

    # Применение логарифмирования и Бокса-Кокса
    ts_data, lambda_val = boxcox(ts_data + 1)
    ts_data = pd.Series(ts_data, index=data[data['unique_id'] == uid]['date'])

    # Проверка на стационарность и разностное преобразование ряда, если необходимо
    p_value = test_stationarity(ts_data)
    if p_value > 0.05:
        ts_data = ts_data.diff(1).dropna()

    # Проверка на тренд и сезонность
    decomposition = seasonal_decompose(ts_data, period=52)
    trend = pd.Series(decomposition.trend)
    seasonality = pd.Series(decomposition.seasonal)
    has_trend = len(trend.dropna().unique()) > 1
    has_seasonality = len(seasonality.dropna().unique()) > 1

    # Обучение модели
    aic, order, best_model = fit_model(ts_data)

    # Предсказание и вычисление метрик, если модель обучена успешно
    if best_model is not None:
        predictions = best_model.predict(start=0, end=len(ts_data)-1, typ='levels')
        predictions = pd.Series(predictions, index=ts_data.index)
        mse = mean_squared_error(ts_data, predictions)
        mae = mean_absolute_error(ts_data, predictions)
        rmse = sqrt(mse)
        r2 = r2_score(ts_data, predictions)

        # Вычисление кастомной бизнес-метрики: error_rate
        error_rate = np.mean(np.abs((predictions - ts_data + 1) / (ts_data + 1)))

        # Проверка нормальности остатков
        residuals = best_model.resid
        normality_test = shapiro(residuals)
        is_normal = normality_test.pvalue > 0.05

        # Добавление результатов в DataFrame
        results_df = results_df.append({
            'unique_id': uid,
            'aic': aic,
            'order': order,
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'error_rate': error_rate,
            'has_trend': has_trend,
            'has_seasonality': has_seasonality,
            'is_normal': is_normal
        }, ignore_index=True)

# Вывод сводки результатов
results_df.to_csv('arima_modeling_results_v2.csv', index=False)
print("Modeling complete. Results saved to '/mnt/data/arima_modeling_results_v2.csv'.")


Processing Unique IDs: 100%|██████████| 212/212 [05:12<00:00,  1.48s/it]

Modeling complete. Results saved to '/mnt/data/arima_modeling_results_v2.csv'.


In [17]:
results_arima = pd.read_csv('C:/Users/ekart/Liudmila_diploma/arima_modeling_results_v2.csv')

In [19]:
results_arima['is_normal'].value_counts()

False    210
True       2
Name: is_normal, dtype: int64

In [21]:
# арима это модель которая учитывает автокорреляцию в данных и к сожалению остатки все равно не нормальные

In [22]:
import numpy as np
import pandas as pd
from math import sqrt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import yeojohnson, shapiro
import warnings
warnings.filterwarnings("ignore")

# Функция для проверки стационарности
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    return dftest[1]

# Функция для подбора и обучения модели ARIMA на основе критерия AIC
def fit_model(ts):
    best_aic = np.inf
    best_order = None
    best_mdl = None

    pq_range = range(3)
    d_range = range(3)

    for p in pq_range:
        for d in d_range:
            for q in pq_range:
                try:
                    tmp_mdl = ARIMA(ts, order=(p, d, q)).fit()
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (p, d, q)
                        best_mdl = tmp_mdl
                except:
                    continue

    return best_aic, best_order, best_mdl

# Dataframe для хранения результатов
results_df = pd.DataFrame(columns=['unique_id', 'aic', 'order', 'mse', 'mae', 'r2', 'error_rate', 'has_trend', 'has_seasonality', 'is_normal'])

# Обработка каждого уникального идентификатора продукта
unique_ids = data['unique_id'].unique()
for uid in tqdm(unique_ids, desc="Processing Unique IDs"):
    ts_data = data[data['unique_id'] == uid].set_index('date')['y']
    ts_data.index = pd.DatetimeIndex(ts_data.index).to_period('W')

    # Применение преобразования Йо-Джонсона
    ts_data, lambda_val = yeojohnson(ts_data)
    ts_data = pd.Series(ts_data, index=data[data['unique_id'] == uid]['date'])

    # Проверка на стационарность и разностное преобразование ряда, если необходимо
    p_value = test_stationarity(ts_data)
    if p_value > 0.05:
        ts_data = ts_data.diff(1).dropna()

    # Проверка на тренд и сезонность
    decomposition = seasonal_decompose(ts_data, period=52)
    trend = pd.Series(decomposition.trend)
    seasonality = pd.Series(decomposition.seasonal)
    has_trend = len(trend.dropna().unique()) > 1
    has_seasonality = len(seasonality.dropna().unique()) > 1

    # Обучение модели
    aic, order, best_model = fit_model(ts_data)

    # Предсказание и вычисление метрик, если модель обучена успешно
    if best_model is not None:
        predictions = best_model.predict(start=0, end=len(ts_data)-1, typ='levels')
        predictions = pd.Series(predictions, index=ts_data.index)
        mse = mean_squared_error(ts_data, predictions)
        mae = mean_absolute_error(ts_data, predictions)
        rmse = sqrt(mse)
        r2 = r2_score(ts_data, predictions)

        # Вычисление кастомной бизнес-метрики: error_rate
        error_rate = np.mean(np.abs((predictions - ts_data + 1) / (ts_data + 1)))

        # Проверка нормальности остатков
        residuals = best_model.resid
        normality_test = shapiro(residuals)
        is_normal = normality_test.pvalue > 0.05

        # Добавление результатов в DataFrame
        results_df = results_df.append({
            'unique_id': uid,
            'aic': aic,
            'order': order,
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'error_rate': error_rate,
            'has_trend': has_trend,
            'has_seasonality': has_seasonality,
            'is_normal': is_normal
        }, ignore_index=True)

# Вывод сводки результатов
results_df.to_csv('arima_modeling_results_v3.csv', index=False)
print("Modeling complete. Results saved to 'arima_modeling_results_v3.csv'.")


Processing Unique IDs: 100%|██████████| 212/212 [05:10<00:00,  1.46s/it]

Modeling complete. Results saved to 'arima_modeling_results_v3.csv'.


In [31]:
res3 = pd.read_csv('C:/Users/ekart/Liudmila_diploma/arima_modeling_results_v3.csv')

In [32]:
res3

,unique_id,aic,order,mse,mae,r2,error_rate,has_trend,has_seasonality,is_normal,rmse
0,9,164.263495,"(2, 0, 2)",0.167976,0.316565,4.208995e-01,0.399374,True,True,True,0.409849
1,10,141.604723,"(2, 0, 0)",0.147249,0.293050,5.395751e-01,0.412756,True,True,False,0.383731
2,11,263.320709,"(2, 0, 2)",0.339071,0.456653,2.133701e-01,2.227611,True,True,False,0.582298
3,12,606.892063,"(2, 0, 0)",3.740130,1.552604,2.324453e-01,0.167728,True,True,False,1.933942
4,13,730.319208,"(2, 0, 2)",8.538199,2.305422,3.543572e-01,0.277584,True,True,False,2.922020
...,...,...,...,...,...,...,...,...,...,...,...
207,389,-949.390630,"(0, 0, 0)",0.000074,0.002401,-3.325622e-07,0.999866,True,True,False,0.008630
208,390,-53.493297,"(1, 0, 0)",0.038587,0.119433,1.248719e-01,1.103038,True,True,False,0.196436
209,391,-60.410147,"(2, 0, 0)",0.036718,0.136144,5.376105e-01,0.351632,True,True,False,0.191618
210,392,-285.461019,"(2, 0, 2)",0.007368,0.057930,5.985787e-01,0.390917,True,True,False,0.085839


тренд и сезонность остались и остатки все такие же

In [27]:
import numpy as np
import pandas as pd
from math import sqrt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from statsmodels.tsa.seasonal import STL
from scipy.stats import yeojohnson, shapiro
import warnings
warnings.filterwarnings("ignore")

# Функция для проверки стационарности
def test_stationarity(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    return dftest[1]

# Функция для подбора и обучения модели ARIMA на основе критерия AIC
def fit_model(ts):
    best_aic = np.inf
    best_order = None
    best_mdl = None

    pq_range = range(3)
    d_range = range(3)

    for p in pq_range:
        for d in d_range:
            for q in pq_range:
                try:
                    tmp_mdl = ARIMA(ts, order=(p, d, q)).fit()
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (p, d, q)
                        best_mdl = tmp_mdl
                except:
                    continue

    return best_aic, best_order, best_mdl

# Dataframe для хранения результатов
results_df = pd.DataFrame(columns=['unique_id', 'aic', 'order', 'mse', 'mae', 'r2', 'error_rate', 'has_trend', 'has_seasonality', 'is_normal'])

# Обработка каждого уникального идентификатора продукта
unique_ids = data['unique_id'].unique()
for uid in tqdm(unique_ids, desc="Processing Unique IDs"):
    ts_data = data[data['unique_id'] == uid].set_index('date')['y']
    ts_data.index = pd.DatetimeIndex(ts_data.index).to_period('W')

    # STL decomposition to remove trend and seasonality
    stl = STL(ts_data, period=52)
    result = stl.fit()
    detrended = result.resid

    # Apply Yeojohnson Transformation
    detrended, lambda_val = yeojohnson(detrended)
    detrended = pd.Series(detrended, index=ts_data.index)

    # Check stationarity and perform differencing if necessary
    p_value = test_stationarity(detrended)
    if p_value > 0.05:
        detrended = detrended.diff().dropna()

    # Fit the ARIMA model
    aic, order, best_model = fit_model(detrended)

    # Make predictions and calculate metrics if model is trained successfully
    if best_model is not None:
        predictions = best_model.predict(start=0, end=len(detrended)-1)
        predictions = pd.Series(predictions, index=detrended.index[:len(predictions)])
        mse = mean_squared_error(detrended, predictions)
        mae = mean_absolute_error(detrended, predictions)
        rmse = sqrt(mse)
        r2 = r2_score(detrended, predictions)

        # Custom business metric: error_rate
        error_rate = np.mean(np.abs((predictions - detrended + 1) / (detrended + 1)))

        # Check normality of residuals
        residuals = best_model.resid
        normality_test = shapiro(residuals)
        is_normal = normality_test.pvalue > 0.05

        # Add results to DataFrame
        results_df = results_df.append({
            'unique_id': uid,
            'aic': aic,
            'order': order,
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'error_rate': error_rate,
            'has_trend': False,
            'has_seasonality': False,
            'is_normal': is_normal
        }, ignore_index=True)

# Output summary of results
results_df.to_csv('arima_modeling_results_v4.csv', index=False)
print("Modeling complete. Results saved to 'arima_modeling_results_v4.csv'.")


Processing Unique IDs: 100%|██████████| 212/212 [04:04<00:00,  1.15s/it]

Modeling complete. Results saved to 'arima_modeling_results_v4.csv'.


In [28]:
res4 = pd.read_csv('C:/Users/ekart/Liudmila_diploma/arima_modeling_results_v4.csv')

In [29]:
res4

,unique_id,aic,order,mse,mae,r2,error_rate,has_trend,has_seasonality,is_normal,rmse
0,9,1212.890954,"(2, 1, 2)",260.127107,7.062473,0.656403,4.612287,False,False,False,16.128456
1,10,1231.120405,"(1, 0, 1)",284.244194,9.346315,0.635296,0.607709,False,False,False,16.859543
2,11,1231.269039,"(2, 1, 2)",295.636200,11.256312,0.519980,1.468012,False,False,False,17.194075
3,12,2002.476873,"(0, 0, 2)",60319.582712,182.892712,0.418514,2.661126,False,False,False,245.600453
4,13,1810.850535,"(2, 1, 2)",17058.586551,93.166211,0.034661,1.764703,False,False,False,130.608524
...,...,...,...,...,...,...,...,...,...,...,...
207,389,1324.813046,"(2, 0, 1)",536.285168,13.184394,0.675826,0.849607,False,False,False,23.157832
208,390,1416.860943,"(2, 2, 1)",1154.907371,16.870021,0.345816,1.082889,False,False,False,33.983928
209,391,1335.447449,"(2, 1, 2)",613.094903,13.980359,0.661053,1.300890,False,False,False,24.760753
210,392,1427.984561,"(2, 1, 2)",1166.678569,18.875456,0.457123,0.995976,False,False,False,34.156677


Удаление тренда и сезонности с помощью STL-декомпозиции:
STL-декомпозиция обеспечивает более гибкое разделение тренда, сезонности и остатков

от сезонности и тренда удалось избавиться, но остатки не вышли нормальными